In [6]:
# All the Includes

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon, mannwhitneyu

In [2]:
# Throwaway Detection

submissions = pd.read_csv('../dataframes/praw_redditor_submissions.csv')
unique_authors = pd.read_csv('../dataframes/unique_authors.csv')

# First Submission of the User we have is posted in r/conspiracy -> supposedly a ThrowAway Account
# Is a bit of a flawed definition because PRAW only allows for new/hot and only 1000k subs/coms.

submissions = submissions.sort_values(by='created_utc', ascending=True)
submissions = submissions.loc[:, ~submissions.columns.str.match('Unnamed')]
submissions = submissions.drop_duplicates(subset='author_name', keep='first')
throwaways = submissions[(submissions['subreddit'] == 'conspiracy')]
throwaways = throwaways.reset_index()
throwaways

,index,id,name,created_utc,title,selftext,score,upvote_ratio,over_18,num_comments,author_id,author_name,subreddit
0,12018,fgqbfr,t3_fgqbfr,1.583895e+09,Super crazy viruses could be released anytime ...,Nukes are starting to look friendly. Virus ca...,2.0,0.56,0.0,7.0,none,AddventureThyme,conspiracy
1,3579,fh3734,t3_fh3734,1.583956e+09,Bill Gates flew on The Lolita Express and also...,NaN,9.0,0.59,0.0,4.0,none,Liquidrome,conspiracy
2,11097,fil7nd,t3_fil7nd,1.584204e+09,"The virus is real, but this here is a drill. A...",Just a theory that while the virus may be real...,87.0,0.88,0.0,31.0,none,Joe_LeFlores,conspiracy
3,15138,fkoxye,t3_fkoxye,1.584536e+09,Killing Joke - I Am The Virus 9.11,[https://www.youtube.com/watch?v=A4wdbibV3IM](...,6.0,0.72,0.0,2.0,none,SexualDeth5quad,conspiracy
4,806,fmjydb,t3_fmjydb,1.584817e+09,Mind Games: Coordinated Attack via COVID-19.1,NaN,7.0,0.89,0.0,3.0,none,4esterField,conspiracy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,439,zxudt9,t3_zxudt9,1.672285e+09,Justice demands correction of military records...,Justice demands correction of military records...,28.0,0.71,0.0,7.0,none,GeneralCarlosQ17,conspiracy
153,28783,zxysq1,t3_zxysq1,1.672298e+09,Moon Landing,Im going down a deep rabbit hole of the moon l...,0.0,0.28,0.0,31.0,none,LifeguardFlat2887,conspiracy
154,29203,zy4cxe,t3_zy4cxe,1.672317e+09,Romania Underground Alien Base photos,NaN,114.0,0.65,0.0,88.0,none,1SPE,conspiracy
155,29730,zy84r0,t3_zy84r0,1.672328e+09,"YouTube Is hiding ""Alex Jones returns"" on JRE ...","""Alex Jones returns"" w Eddie bravo is 4 hrs 4...",0.0,0.50,0.0,22.0,none,TryhardNobody,conspiracy


In [3]:
# Throwaway Analysis

normal_users = submissions[submissions['author_name'].isin(throwaways['author_name']) == False]
normal_users = normal_users.drop_duplicates(subset="author_name", keep="first")
normal_users = normal_users.reset_index()
max_users = len(normal_users['author_name'])+ len(throwaways['author_name'])
percentage_ta = round((len(throwaways['author_name'])/len(normal_users['author_name'])) * 100, 2)
print("Considering our definition for a Throwaway Account we conclude that " +
      str(percentage_ta) + "% of our "
+ str(max_users) + " potentially use Throwaway Accounts on the Subreddit r/conspiracy.")

Considering our definition for a Throwaway Account we conclude that 38.57% of our 564 potentially use Throwaway Accounts on the Subreddit r/conspiracy.


In [23]:
# -------------------------------------------------------------------------------
# MWU(#Subs-TA, #Subs-N-TA) - Unterschied zwischen Throw Aways und Normalen Usern
# -------------------------------------------------------------------------------
submissions = pd.read_csv('../dataframes/praw_redditor_submissions.csv')
throwaways = submissions[submissions['author_name'].isin(throwaways['author_name']) == True]
normal_users = submissions[submissions['author_name'].isin(throwaways['author_name']) == False]

# Shuffle the Samples
normal_users = normal_users.sample(frac=1)
throwaways = throwaways.sample(frac=1)

throwaways_mwu = throwaways.groupby(['author_name']).agg(number_of_submissions=('subreddit', 'count'))
normal_users_mwu = normal_users.groupby(['author_name']).agg(number_of_submissions=('subreddit', 'count'))

mwu_result = mannwhitneyu(throwaways_mwu, normal_users_mwu, alternative='two-sided')
print("If the p-value is less than or equal to the significance level, the decision is to reject the null hypothesis.\n"
      + "Usually, a significance level (denoted as α or alpha) of 0.05 works well." + "Comparing our two samples (Throwaways and Normal users), we arrive at a p-value of "
      + str(np.round_(mwu_result.pvalue[0], decimals=4)) + ", thus we reject and there is no significant difference between the two groups."
)

If the p-value is less than or equal to the significance level, the decision is to reject the null hypothesis.
Usually, a significance level (denoted as α or alpha) of 0.05 works well.Comparing our two samples (Throwaways and Normal users), we arrive at a p-value of 0.4893, thus we reject and there is no significant difference between the two groups.
